#### Dates and Timestamps

In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
from pyspark.sql import SparkSession, Row
import pyspark.sql.functions as F
from pyspark.sql import types as T

spark = SparkSession.builder.appName("Dates and Timestamps").getOrCreate()

<IPython.core.display.Javascript object>

In [3]:
spark

<IPython.core.display.Javascript object>

Create a dataframe with different date formats as strings.

In [4]:
rows = [
    Row("2020-01-03"),
    Row("2020 01 10"),
    Row("2020 Jan 10"),
        Row("Sat, 11 Jan 2020"),
]

myrdd = spark.sparkContext.parallelize(rows)

schema = T.StructType(
    [
        T.StructField(name="date_str", dataType=T.StringType(),nullable=True)]
)
df = spark.createDataFrame(myrdd, schema)


<IPython.core.display.Javascript object>

In [5]:
df.printSchema()

root
 |-- date_str: string (nullable = true)



<IPython.core.display.Javascript object>

SimpleDateFormat is a concrete class for formatting and parsing dates in a locale-sensitive manner.
You can use the patterns below to convert a string to a date:

https://docs.oracle.com/javase/7/docs/api/java/text/SimpleDateFormat.html

Below we use some of these patterns to transform the above strings to dates.

In [6]:
df = df.withColumn(
    "date",
    F.when(
        F.to_date(F.col("date_str"), "yyyy-MM-dd").isNotNull(),
        F.to_date(F.col("date_str"), "yyyy-MM-dd"),
    ).otherwise(
        F.when(
            F.to_date(F.col("date_str"), "yyyy MM dd").isNotNull(),
            F.to_date(F.col("date_str"), "yyyy MM dd"),
        ).otherwise(
            F.when(
                F.to_date(F.col("date_str"), "yyyy MMMM dd").isNotNull(),
                F.to_date(F.col("date_str"), "yyyy MMMM dd"),
            ).otherwise(
                F.when(
                    F.to_date(F.col("date_str"), "E, dd MMMM yy").isNotNull(),
                    F.to_date(F.col("date_str"), "E, dd MMMM yy"),
                )
            ),
        ),
    ),
)

<IPython.core.display.Javascript object>

We can also use *to_timestamp* instead of *to_date* to transform to timestamps.

In [7]:
df = df.withColumn(
    "timestamp",
    F.when(
        F.to_timestamp(F.col("date_str"), "yyyy-MM-dd").isNotNull(),
        F.to_timestamp(F.col("date_str"), "yyyy-MM-dd"),
    ).otherwise(
        F.when(
            F.to_timestamp(F.col("date_str"), "yyyy MM dd").isNotNull(),
            F.to_timestamp(F.col("date_str"), "yyyy MM dd"),
        ).otherwise(
            F.when(
                F.to_timestamp(F.col("date_str"), "yyyy MMMM dd").isNotNull(),
                F.to_timestamp(F.col("date_str"), "yyyy MMMM dd"),
            ).otherwise(
                F.when(
                    F.to_timestamp(F.col("date_str"), "E, dd MMMM yy").isNotNull(),
                    F.to_timestamp(F.col("date_str"), "E, dd MMMM yy"),
                )
            ),
        ),
    ),
)

<IPython.core.display.Javascript object>

In [8]:
df.printSchema()

root
 |-- date_str: string (nullable = true)
 |-- date: date (nullable = true)
 |-- timestamp: timestamp (nullable = true)



<IPython.core.display.Javascript object>

In [9]:
df.show()

+----------------+----------+-------------------+
|        date_str|      date|          timestamp|
+----------------+----------+-------------------+
|      2020-01-03|2020-01-03|2020-01-03 00:00:00|
|      2020 01 10|2020-01-10|2020-01-10 00:00:00|
|     2020 Jan 10|2020-01-10|2020-01-10 00:00:00|
|Sat, 11 Jan 2020|2020-01-11|2020-01-11 00:00:00|
+----------------+----------+-------------------+



<IPython.core.display.Javascript object>

Let's subtract 10 days from the date column.

In [10]:
df.withColumn("date_sub_10", F.date_sub("date", 10)).show()

+----------------+----------+-------------------+-----------+
|        date_str|      date|          timestamp|date_sub_10|
+----------------+----------+-------------------+-----------+
|      2020-01-03|2020-01-03|2020-01-03 00:00:00| 2019-12-24|
|      2020 01 10|2020-01-10|2020-01-10 00:00:00| 2019-12-31|
|     2020 Jan 10|2020-01-10|2020-01-10 00:00:00| 2019-12-31|
|Sat, 11 Jan 2020|2020-01-11|2020-01-11 00:00:00| 2020-01-01|
+----------------+----------+-------------------+-----------+



<IPython.core.display.Javascript object>

Let's add 20 days to the date column.

In [11]:
df.withColumn("date_add_10", F.date_add("date", 20)).show()

+----------------+----------+-------------------+-----------+
|        date_str|      date|          timestamp|date_add_10|
+----------------+----------+-------------------+-----------+
|      2020-01-03|2020-01-03|2020-01-03 00:00:00| 2020-01-23|
|      2020 01 10|2020-01-10|2020-01-10 00:00:00| 2020-01-30|
|     2020 Jan 10|2020-01-10|2020-01-10 00:00:00| 2020-01-30|
|Sat, 11 Jan 2020|2020-01-11|2020-01-11 00:00:00| 2020-01-31|
+----------------+----------+-------------------+-----------+



<IPython.core.display.Javascript object>

If we want to see the difference in days between today and the date column.

In [12]:
df.withColumn("date_diff", F.datediff(F.current_date(), "date")).show()

+----------------+----------+-------------------+---------+
|        date_str|      date|          timestamp|date_diff|
+----------------+----------+-------------------+---------+
|      2020-01-03|2020-01-03|2020-01-03 00:00:00|       10|
|      2020 01 10|2020-01-10|2020-01-10 00:00:00|        3|
|     2020 Jan 10|2020-01-10|2020-01-10 00:00:00|        3|
|Sat, 11 Jan 2020|2020-01-11|2020-01-11 00:00:00|        2|
+----------------+----------+-------------------+---------+



<IPython.core.display.Javascript object>

We might need to create three new features, for year, month and day of week respectively.

* From the example below you can see the day of week starts on Sunday, with Sunday being 0 and Saturday being 7.

In [13]:
df.withColumn("year", F.year("date")).withColumn("month", F.month("date")).withColumn(
    "day", F.dayofweek("date")
).show()

+----------------+----------+-------------------+----+-----+---+
|        date_str|      date|          timestamp|year|month|day|
+----------------+----------+-------------------+----+-----+---+
|      2020-01-03|2020-01-03|2020-01-03 00:00:00|2020|    1|  6|
|      2020 01 10|2020-01-10|2020-01-10 00:00:00|2020|    1|  6|
|     2020 Jan 10|2020-01-10|2020-01-10 00:00:00|2020|    1|  6|
|Sat, 11 Jan 2020|2020-01-11|2020-01-11 00:00:00|2020|    1|  7|
+----------------+----------+-------------------+----+-----+---+



<IPython.core.display.Javascript object>

Now that we have our dates/timestamps in the correct format we can easily create filters.

In [14]:
df.where(F.col("date") > "2020-01-10").show()

+----------------+----------+-------------------+
|        date_str|      date|          timestamp|
+----------------+----------+-------------------+
|Sat, 11 Jan 2020|2020-01-11|2020-01-11 00:00:00|
+----------------+----------+-------------------+



<IPython.core.display.Javascript object>

The two examples below show that Spark does pretty well sometimes by reading dates or times even with different formats but many times you won't have such an easy task, having strangely formatted data.

In [15]:
df.where(F.col("date_str") >= "2020 01 10").show()

+----------------+----------+-------------------+
|        date_str|      date|          timestamp|
+----------------+----------+-------------------+
|      2020-01-03|2020-01-03|2020-01-03 00:00:00|
|      2020 01 10|2020-01-10|2020-01-10 00:00:00|
|     2020 Jan 10|2020-01-10|2020-01-10 00:00:00|
|Sat, 11 Jan 2020|2020-01-11|2020-01-11 00:00:00|
+----------------+----------+-------------------+



<IPython.core.display.Javascript object>

In [16]:
df.where(F.col("date_str") > "2020 01 10").show()

+----------------+----------+-------------------+
|        date_str|      date|          timestamp|
+----------------+----------+-------------------+
|      2020-01-03|2020-01-03|2020-01-03 00:00:00|
|     2020 Jan 10|2020-01-10|2020-01-10 00:00:00|
|Sat, 11 Jan 2020|2020-01-11|2020-01-11 00:00:00|
+----------------+----------+-------------------+



<IPython.core.display.Javascript object>